In [30]:
from langgraph.graph import StateGraph, START, END
from typing import TypedDict, Annotated
from langchain_core.messages import BaseMessage, HumanMessage, AIMessage
from langchain_google_genai import ChatGoogleGenerativeAI
from langgraph.checkpoint.memory import MemorySaver

from dotenv import load_dotenv
load_dotenv()

True

In [31]:
from langgraph.graph.message import add_messages

class ChatState(TypedDict):
    messages: Annotated[list[BaseMessage], add_messages]

In [32]:
chat_model = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash",
    temperature=0.7,
    max_tokens=1000
)

def chat_node(state: ChatState) -> ChatState:
    messages = state['messages'] if state['messages'] else [HumanMessage(content="Hello!")]

    ai_response = chat_model.invoke(messages)

    return {
        'messages': messages + [AIMessage(content=ai_response.content)]
    }

In [35]:
checkpointer = MemorySaver()
graph = StateGraph(ChatState)

graph.add_node('chat_node', chat_node)

graph.add_edge(START, 'chat_node')
graph.add_edge('chat_node', END)

chatbot = graph.compile(checkpointer=checkpointer)

In [36]:
thread_id='1'
while True:
    user_input = input("Type here: ")
    
    print(f"You: {user_input}")
    if user_input.strip().lower() in ['exit', 'quit']:
        print("Exiting the chat.")
        break
    
    config = {'configurable': {'thread_id': thread_id}}
    response = chatbot.invoke({
        'messages': [HumanMessage(content=user_input)]}, config=config)
    
    print('AI: ' + response['messages'][-1].content)
    

You: hi I am anubrata
AI: Hello Anubrata!  How can I help you today?
AI: Hello Anubrata!  How can I help you today?
You: what is name?
You: what is name?
AI: I don't have a name. I am a large language model, an AI.
AI: I don't have a name. I am a large language model, an AI.
You: what is my name?
You: what is my name?
AI: Your name is Anubrata.
AI: Your name is Anubrata.
You: exit
Exiting the chat.
You: exit
Exiting the chat.


In [40]:
chatbot.get_state(config=config)

StateSnapshot(values={'messages': [HumanMessage(content='hi I am anubrata', additional_kwargs={}, response_metadata={}, id='b0131a8b-2769-4609-b41d-8f51abf7394c'), AIMessage(content='Hello Anubrata!  How can I help you today?', additional_kwargs={}, response_metadata={}, id='6a369def-75a3-4163-9bb3-ba2aa7a3a9d9'), HumanMessage(content='what is name?', additional_kwargs={}, response_metadata={}, id='f72135f3-d88b-4cf5-817f-b3b346128150'), AIMessage(content="I don't have a name. I am a large language model, an AI.", additional_kwargs={}, response_metadata={}, id='97c7fe1c-79d6-48e5-8c5b-d591f60d0aa6'), HumanMessage(content='what is my name?', additional_kwargs={}, response_metadata={}, id='11fad4f3-84c3-43fb-80ee-df2ce9dc0b70'), AIMessage(content='Your name is Anubrata.', additional_kwargs={}, response_metadata={}, id='7c83fcd6-8d84-491d-b018-42c8d988d59b')]}, next=(), config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f0700c0-e37b-683a-8007-036a1623c276'}